Art. 32 É obrigatória a manutenção de, no mínimo, 0,2 mg/L de cloro residual livre ou 2 mg/L de cloro residual combinado ou de 0,2 mg/L de dióxido de cloro em toda a extensão do sistema de distribuição (reservatório e rede) e nos pontos de consumo.

In [1]:
import os
import re
import sys
import pprint
import pandas as pd
from scipy.stats import gmean
from dateutil.relativedelta import relativedelta

In [2]:
from paths import *

In [3]:
# Parameters
cod_ibge = '3548906' # São Carlos
cod_ibge = '3526902' # Limeira
cod_ibge = '3501608' # Americana

In [4]:
# Read Table
df_bruta = pd.read_excel(
    os.path.join(output_path, str(cod_ibge), 'dados brutos', 'controle', 'controle_mensal_parametros_basicos.xlsx')
)

In [12]:
# Filtra Apenas SAAs
df = df_bruta.loc[df_bruta['Tipo Da Forma De Abastecimento'] == 'SAA']

# Filtra Apenas Último Ano
df = df[df['Ano De Referência'] == max(df['Ano De Referência'])].copy()

In [14]:
df.info()
list(df.columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 327586 to 328329
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Região Geográfica                  744 non-null    object 
 1   Regional De Saúde                  744 non-null    object 
 2   Município                          744 non-null    object 
 3   Tipo Da Instituição                744 non-null    object 
 4   Sigla Da Instituição               0 non-null      float64
 5   Nome Da Instituição                744 non-null    object 
 6   Cnpj Da Instituição                744 non-null    float64
 7   Nome Do Escritório Regional/Local  0 non-null      float64
 8   Cnpj Do Escritório Regional/Local  0 non-null      float64
 9   Tipo Da Forma De Abastecimento     744 non-null    object 
 10  Código Forma De Abastecimento      744 non-null    object 
 11  Nome Da Forma De Abastecimento     744 non-null   

['Região Geográfica',
 'Regional De Saúde',
 'Município',
 'Tipo Da Instituição',
 'Sigla Da Instituição',
 'Nome Da Instituição',
 'Cnpj Da Instituição',
 'Nome Do Escritório Regional/Local',
 'Cnpj Do Escritório Regional/Local',
 'Tipo Da Forma De Abastecimento',
 'Código Forma De Abastecimento',
 'Nome Da Forma De Abastecimento',
 'Nome Da Eta / Uta',
 'Tipo De Filtração',
 'Ano De Referência',
 'Mês De Referência',
 'Ponto De Monitoramento',
 'Parâmetro',
 'Campo',
 'Valor',
 'Uf',
 'Código Ibge']

<br>

# Artigo 32

In [15]:
set(df['Parâmetro'])

{'Bactérias Heterotróficas (UFC/mL)',
 'Cloro Residual Livre (mg/L)',
 'Coliformes totais',
 'Cor (uH)',
 'Escherichia coli',
 'Fluoreto (mg/L)',
 'Turbidez (uT)',
 'pH'}

In [16]:
#df = df[df['Parâmetro'] == 'Escherichia coli'].copy()
df = df[df['Parâmetro'].str.contains('Cloro')].copy()
df.head()

,Região Geográfica,Regional De Saúde,Município,Tipo Da Instituição,Sigla Da Instituição,Nome Da Instituição,Cnpj Da Instituição,Nome Do Escritório Regional/Local,Cnpj Do Escritório Regional/Local,Tipo Da Forma De Abastecimento,...,Nome Da Eta / Uta,Tipo De Filtração,Ano De Referência,Mês De Referência,Ponto De Monitoramento,Parâmetro,Campo,Valor,Uf,Código Ibge
327619,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),Número de amostras analisadas,2436,SP,350160
327620,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados > 5,0 mg/L",0,SP,350160
327621,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados > 2,0 mg/L e <= 5,0mg/L",223,SP,350160
327622,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados >= 0,2 mg/L e <= 2,0mg/L",2213,SP,350160
327623,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados < 0,2 mg/L",0,SP,350160


In [17]:
set(df['Ponto De Monitoramento'])

{'SAÍDA DO TRATAMENTO', 'SISTEMA DE DISTRIBUIÇÃO'}

In [18]:
df = df[df['Ponto De Monitoramento'] == 'SAÍDA DO TRATAMENTO'].copy()
df.head()

,Região Geográfica,Regional De Saúde,Município,Tipo Da Instituição,Sigla Da Instituição,Nome Da Instituição,Cnpj Da Instituição,Nome Do Escritório Regional/Local,Cnpj Do Escritório Regional/Local,Tipo Da Forma De Abastecimento,...,Nome Da Eta / Uta,Tipo De Filtração,Ano De Referência,Mês De Referência,Ponto De Monitoramento,Parâmetro,Campo,Valor,Uf,Código Ibge
327619,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),Número de amostras analisadas,2436,SP,350160
327620,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados > 5,0 mg/L",0,SP,350160
327621,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados > 2,0 mg/L e <= 5,0mg/L",223,SP,350160
327622,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados >= 0,2 mg/L e <= 2,0mg/L",2213,SP,350160
327623,SUDESTE,GVS XVII - CAMPINAS,AMERICANA,Local,NaN,DEPARTAMENTO DE AGUA E ESGOTO,4.675569e+13,NaN,NaN,SAA,...,ETA,FILTRAÇÃO RÁPIDA,2021,11,SAÍDA DO TRATAMENTO,Cloro Residual Livre (mg/L),"Número de dados < 0,2 mg/L",0,SP,350160


In [19]:
df = df[['Ano De Referência', 'Mês De Referência', 'Campo', 'Valor']].copy()
df = df.sort_values(by=['Ano De Referência', 'Mês De Referência', 'Campo']).copy()
df.head()

,Ano De Referência,Mês De Referência,Campo,Valor
327648,2021,1,Número de amostras analisadas,1483
327652,2021,1,"Número de dados < 0,2 mg/L",0
327650,2021,1,"Número de dados > 2,0 mg/L e <= 5,0mg/L",0
327649,2021,1,"Número de dados > 5,0 mg/L",0
327651,2021,1,"Número de dados >= 0,2 mg/L e <= 2,0mg/L",1483


Americana não tinha amostras no Ponto de captação....

{'SAÍDA DO TRATAMENTO', 'SISTEMA DE DISTRIBUIÇÃO'}

In [23]:
df['Valor'] = df['Valor'].astype(str).str.replace(',','.')
df['Valor'] = df['Valor'].astype(float).fillna(0.0)
df.head()

,Ano De Referência,Mês De Referência,Campo,Valor
327648,2021,1,Número de amostras analisadas,1483.0
327652,2021,1,"Número de dados < 0,2 mg/L",0.0
327650,2021,1,"Número de dados > 2,0 mg/L e <= 5,0mg/L",0.0
327649,2021,1,"Número de dados > 5,0 mg/L",0.0
327651,2021,1,"Número de dados >= 0,2 mg/L e <= 2,0mg/L",1483.0


<br>

Fazer gráfico multidimensional...
Gráfico...